In [1]:
From Coq Require Import ZArith Reals Psatz.
From Flocq Require Import Core Plus_error.

Open Scope R_scope.

Cell evaluated. 
 

 
 
 Cell rolled back. 
 

 
 
 
 Rollback cell 
 

 
 
 Auto rollback

In [35]:
Definition leapfrog_1 x0 v0 h F := 
    let v12 := v0 + 0.5*  h* F x0 in
    let x1  := x0 + h* v0 in
    pair x1 v12.
    
Definition leapfrog x0 v0 h F := 
    let v12 := snd (leapfrog_1 x0 v0 h F) in
    let x1  := fst (leapfrog_1 x0 v0 h F) in
    let v1  := v12 + 0.5* h* F x1 in
    pair x1 v1.

Cell evaluated. 
 

 
 
 Cell rolled back. 
 

 
 
 
 Rollback cell 
 

 
 
 Auto rollback

In [37]:
Definition id (x:R) := x.

Lemma leapfrog_gt:
     forall x0 v0 h,
     Rgt x0 0 -> Rgt v0 0 -> Rgt h 0 ->
     Rgt (fst (leapfrog_1 x0 v0 h id)) 0. 

Proof.
intros x0 v0 h Hx Hv Hh.
unfold leapfrog_1.
unfold fst. 
assert (H1 : Rlt 0 (h * v0)).
replace 0 with (h * 0).
apply (Rmult_lt_compat_l h 0 v0).
unfold Rgt in Hh; auto with real. 
unfold Rgt in Hv; auto with real.
auto with real.
unfold Rgt.
unfold Rgt in Hx.
assert (H2 : Rlt (x0 + 0) (x0 + h * v0)).
apply (Rplus_lt_compat_l x0 0 (h*v0)); auto with real.
(replace (x0 +0) with (x0) in H2); auto with real.
apply (Rlt_trans 0 x0 (x0 + h * v0)); auto with real.
Qed.

Cell evaluated. 
 

 
 
 Cell rolled back. 
 

 
 
 
 Rollback cell 
 

 
 
 Auto rollback